In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
df = pd.read_csv('15stocks_price.csv')
df.set_index('date_gsheets', inplace=True)

In [3]:
df.head()

,AAPL,KO,NFLX,BRK.B,DIS,IBM,VZ,WMT,GE,TSLA,MA,AMZN,MSFT,UN,V
date_gsheets,,,,,,,,,,,,,,,
01/03/2012 16:00,58.75,35.07,10.32,77.68,38.31,186.30,39.73,60.33,18.36,28.08,36.84,179.03,26.77,34.92,25.75
01/04/2012 16:00,59.06,34.85,11.49,76.80,38.85,185.54,39.21,59.71,18.56,27.71,35.63,177.51,27.40,34.45,25.29
01/05/2012 16:00,59.72,34.69,11.33,76.93,39.50,184.66,38.94,59.42,18.55,27.12,35.24,177.61,27.68,34.49,25.48
01/06/2012 16:00,60.34,34.47,12.33,76.39,39.91,182.54,38.33,59.00,18.65,26.91,34.29,182.61,28.11,33.27,25.18
01/09/2012 16:00,60.25,34.47,14.03,76.29,39.75,181.59,38.37,59.18,18.86,27.25,34.58,178.56,27.74,33.72,24.98


In [5]:
returns_df = df.pct_change(1).dropna()

In [6]:
returns_df

,AAPL,KO,NFLX,BRK.B,DIS,IBM,VZ,WMT,GE,TSLA,MA,AMZN,MSFT,UN,V
date_gsheets,,,,,,,,,,,,,,,
01/04/2012 16:00,0.005277,-0.006273,0.113372,-0.011329,0.014096,-0.004079,-0.013088,-0.010277,0.010893,-0.013177,-0.032845,-0.008490,0.023534,-0.013459,-0.017864
01/05/2012 16:00,0.011175,-0.004591,-0.013925,0.001693,0.016731,-0.004743,-0.006886,-0.004857,-0.000539,-0.021292,-0.010946,0.000563,0.010219,0.001161,0.007513
01/06/2012 16:00,0.010382,-0.006342,0.088261,-0.007019,0.010380,-0.011481,-0.015665,-0.007068,0.005391,-0.007743,-0.026958,0.028152,0.015535,-0.035373,-0.011774
01/09/2012 16:00,-0.001492,0.000000,0.137875,-0.001309,-0.004009,-0.005204,0.001044,0.003051,0.011260,0.012635,0.008457,-0.022178,-0.013163,0.013526,-0.007943
01/10/2012 16:00,0.003485,0.005802,-0.024234,0.014812,-0.003019,-0.001542,0.005212,-0.002366,-0.007423,0.013578,0.008676,0.004368,0.003605,0.005635,-0.001201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/22/2017 16:00:00,0.000000,-0.000219,0.006998,-0.004877,-0.008214,0.006601,0.003396,0.001530,0.001717,-0.019478,0.002126,-0.005448,0.000117,0.001599,0.002491
12/26/2017 16:00:00,-0.025370,0.004826,-0.011477,-0.001768,-0.005061,0.002164,0.000564,0.009673,-0.004000,-0.024323,0.002851,0.007190,-0.001286,0.002661,0.002662
12/27/2017 16:00:00,0.000176,0.002620,-0.008095,0.005669,-0.004440,0.001963,0.001127,0.001008,-0.002869,-0.017807,0.000793,0.004674,0.003630,-0.001415,0.009116


In [7]:
def getPortReturn(weights):
    exp_ret_portfolio = np.dot(np.transpose(weights), returns_df.mean()) * 250
    
    return exp_ret_portfolio

In [8]:
num_stocks = len(returns_df.columns)
init_weights = [1 / num_stocks] * num_stocks

In [9]:
init_weights

[0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667]

In [11]:
getPortReturn(init_weights)

0.20535127905081593

In [12]:
target_return = 0.3

In [13]:
bounds = tuple((0,1) for i in range(num_stocks))

In [14]:
bounds

((0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1))

In [15]:
cons = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1},
       {'type' : 'eq', 'fun' : lambda x : x.dot(returns_df.mean()) * 250 - target_return})

In [16]:
results = minimize(fun=getPortReturn, x0=init_weights, bounds=bounds, constraints=cons)

In [17]:
results

     fun: 0.300000000025853
     jac: array([ 0.20626795,  0.05398266,  0.61322932,  0.16528047,  0.18761196,
       -0.01560373,  0.0603794 ,  0.09600191,  0.00856303,  0.51433235,
        0.2549938 ,  0.35404797,  0.2169628 ,  0.09711015,  0.26710914])
 message: 'Optimization terminated successfully'
    nfev: 32
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.06686058, 0.03464624, 0.15294893, 0.05819011, 0.0629141 ,
       0.01992598, 0.03599941, 0.04353497, 0.02503821, 0.13202833,
       0.07716801, 0.09812187, 0.06912296, 0.04376941, 0.07973089])

In [18]:
getPortReturn(weights=results['x'])

0.300000000025853

In [19]:
optimised_weights = pd.DataFrame(results['x'])

In [20]:
optimised_weights

,0
0,0.066861
1,0.034646
2,0.152949
3,0.058190
4,0.062914
5,0.019926
6,0.035999
7,0.043535
8,0.025038
9,0.132028


In [21]:
optimised_weights.index = returns_df.columns

In [22]:
optimised_weights

,0
AAPL,0.066861
KO,0.034646
NFLX,0.152949
BRK.B,0.058190
DIS,0.062914
IBM,0.019926
VZ,0.035999
WMT,0.043535
GE,0.025038
TSLA,0.132028
